In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression, LinearRegression

iris = load_iris()
X=iris.data
y=iris.target

1. Изучите разбиение Leave-P-Out. Продемонстрируйте работу этого алгоритма на примере из лабораторной работы.

In [ ]:
from sklearn.model_selection import LeavePOut, cross_val_score, cross_validate

In [ ]:
lpo = LeavePOut(2)

lpo.get_n_splits(X)

for i, (train_index, test_index) in enumerate(lpo.split(X)):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  #print(f"Fold {i}:")
  #print(f"  Train: index={train_index}")
  #print(f"  Test:  index={test_index}")

model = LogisticRegression(solver='liblinear')
cv_results = cross_val_score(model,                  # модель
                             X,                      # матрица признаков
                             y,                      # вектор цели
                             cv = lpo,           # тип разбиения
                             scoring = 'f1_macro',   # метрика
                             n_jobs=-1)              # используются все ядра CPU

print("Кросс-валидация: ", cv_results)
print("Среднее по кросс-валидации: ", cv_results.mean())

Кросс-валидация:  [1. 1. 1. ... 1. 1. 1.]
Среднее по кросс-валидации:  0.9405219985085758


2. Изучите функцию cross_validate(). Продемонстрируйте работу этой функции на тех же данных.

In [ ]:
cv_results_2 = cross_validate(model, # модель для обучения
                            X, y, # Данные
                            scoring='f1_macro',
                            cv=5) # количество фолдов

# выводит словарь с описанием времени обучения, рассчета ошибки
  # и результаты ошибок на каждой группе тестовых данных

for k in cv_results_2.keys():
  print(f'{k}: {cv_results_2[k]}')
print("Среднее по кросс-валидации: ", cv_results_2['test_score'].mean())

fit_time: [0.00197458 0.00143623 0.00108314 0.00105691 0.00107169]
score_time: [0.00248504 0.00141191 0.00141668 0.00140071 0.00133562]
test_score: [1.         0.96658312 0.93333333 0.89769821 1.        ]
Среднее по кросс-валидации:  0.959522933505973


3. Оцените при помощи кросс-валидации другие метрики эффективности для этой же модели.

In [ ]:
cv_results_3 = cross_validate(model, X, y,
                             scoring=['accuracy', 'f1_micro', 'f1_macro', 'f1_weighted', 'precision_weighted', 'recall_weighted'],
                             cv=5)

_ = [print(f' Среднее значение {k} : {cv_results_3[k].mean().round(3)}') for k in cv_results_3.keys()]

 Среднее значение fit_time : 0.139
 Среднее значение score_time : 0.011
 Среднее значение test_accuracy : 0.967
 Среднее значение test_f1_micro : 0.967
 Среднее значение test_f1_macro : 0.966
 Среднее значение test_f1_weighted : 0.966
 Среднее значение test_precision_weighted : 0.971
 Среднее значение test_recall_weighted : 0.967


4. Сравните кросс-валидированные результаты работы нескольких моделей на одних и тех же данных.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPClassifier

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
models = {"Метод опорных векторов": SVC(),
          "Перцептрон": MLPClassifier(),
          "Деревья решений": DecisionTreeClassifier(),
          "Метод k ближайших соседей": KNeighborsClassifier(),
          "Гауссовский Наивный Байес": GaussianNB(),
          "Случайный лес" :RandomForestClassifier()
          }

for name, model in models.items():
  cv_results_4 = cross_validate(model, X, y,
                             scoring=['f1_weighted'],
                             cv=5)

  _ = [print(f' Среднее значение {k} модели {name}  : {cv_results_4[k].mean().round(3)}') for k in ['test_f1_weighted']]

 Среднее значение test_f1_weighted модели Метод опорных векторов  : 0.967
 Среднее значение test_f1_weighted модели Перцептрон  : 0.987
 Среднее значение test_f1_weighted модели Деревья решений  : 0.967
 Среднее значение test_f1_weighted модели Метод k ближайших соседей  : 0.973
 Среднее значение test_f1_weighted модели Гауссовский Наивный Байес  : 0.953
 Среднее значение test_f1_weighted модели Случайный лес  : 0.966


5. Повторите анализ на другом датасете: встроенном наборе данных о диабете.

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

In [ ]:
diabetes = load_diabetes()
X_d = diabetes.data
y_d = diabetes.target

metrics_MSE = []
metrics_r2 = []
model_d = SVR()

In [ ]:
y_d[:10]

array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310.])

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold, LeaveOneOut

In [ ]:
kf = KFold(n_splits = 5,shuffle=True, random_state=15)

for i, (train_index, test_index) in enumerate(kf.split(y_d)):
  X_train, X_test = X_d[train_index], X_d[test_index]
  y_train, y_test = y_d[train_index], y_d[test_index]

  model_d.fit(X_train, y_train)
  y_pred = model_d.predict(X_test)

  metrics_MSE.append(mean_squared_error(y_test, y_pred))
  metrics_r2.append(r2_score(y_test, y_pred))

print("Среднее r2 по кросс-валидации: ", np.array(metrics_r2).mean().round(3))
print("Среднее MSE по кросс-валидации: ", np.array(metrics_MSE).mean().round(3))

Среднее r2 по кросс-валидации:  0.15
Среднее MSE по кросс-валидации:  5020.889


In [ ]:
methods = {"k_fold разбиение" : KFold(n_splits = 3,shuffle=True, random_state=15),
           "Stratified k-Fold" : StratifiedKFold(n_splits=3,shuffle=True, random_state=15),
           #"Leave One Out" : LeaveOneOut()
           }

In [ ]:
def cross_val(X, y, model, methods=methods):
  for name, method in methods.items():
    cv_results_5 = cross_val_score(model,
                                X,
                                y,
                                cv = method,
                                scoring = 'r2',
                                n_jobs=-1)

    print(f"{name} \nКросс-валидация:  {cv_results_5}")
    print(f"Среднее по кросс-валидации:  {cv_results_5.mean().round(3)}")
    print(f"Дисперсия по кросс-валидации: {cv_results_5.std().round(3)} \n")

In [ ]:
cross_val(X_d, y_d, model_d)

k_fold разбиение 
Кросс-валидация:  [0.14964348 0.12256466 0.15055728]
Среднее по кросс-валидации:  0.141
Дисперсия по кросс-валидации: 0.013 

Stratified k-Fold 
Кросс-валидация:  [0.13581041 0.13826692 0.14445479]
Среднее по кросс-валидации:  0.14
Дисперсия по кросс-валидации: 0.004 



6. Сделайте k-блочную перекрёстную проверку (KFold) модели логистической регрессии, предварительно стандартизировав данные. Для этого создайте конвейер с помощью make_pipeline из библиотеки sklearn.pipeline, который будет стандартизировать, а затем выполнять логистическую регрессию.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
kf = KFold(n_splits = 5,shuffle=True, random_state=15)

for i, (train_index, test_index) in enumerate(kf.split(y)):
  X6_train, X6_test = X[train_index], X[test_index]
  y6_train, y6_test = y[train_index], y[test_index]

pipe = make_pipeline(StandardScaler(), LogisticRegression())

pipe.fit(X6_train, y6_train)

y6_pred = pipe.predict(X6_test)

print(f"R2-score: {r2_score(y6_test, y6_pred).round(3)}")
print(f"MAPE: {mean_absolute_percentage_error(y6_test, y6_pred).round(3)}")

R2-score: 0.905
MAPE: 0.033
